<a href="https://colab.research.google.com/github/alucinationx/Coursera_Capstone/blob/main/finalcapstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import libraries
!pip install geocoder
!pip install foursquare
import pandas as pd
import numpy as np
import requests 
from pandas.io.json import json_normalize 
import folium 
import geocoder
print('everything is fine')

     |████████████████████████████████| 102kB 3.9MB/s 
everything is fine


In [2]:
# Define Foursquare Credentials and Version

CLIENT_ID = 'W5FPHFGBRSQIAUCCW51OJVX32SVS1RLAMZN1JRWXBR2TDXB0' # your Foursquare ID
CLIENT_SECRET = 'RELN10XSC1GQDZZX0CLOZY3E2LL12EG33E2P0BSFEO00K3K2' # your Foursquare Secret
VERSION = '20201205' # Foursquare API version
LIMIT = 40

In [3]:
import requests

request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20201205',
    "section": "drinks",
    "near": "Bauru",
    "radius": 10000,
    "limit": 50}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

In [4]:
d = data.json()["response"]
d.keys()

dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'query', 'totalResults', 'suggestedBounds', 'groups'])

In [5]:
d["headerLocationGranularity"], d["headerLocation"], d["headerFullLocation"]

('city', 'Bauru', 'Bauru')

In [6]:
d["suggestedBounds"], d["totalResults"]

({'ne': {'lat': -22.312547858979208, 'lng': -49.02842730721986},
  'sw': {'lat': -22.373439135701844, 'lng': -49.10085438487687}},
 47)

In [7]:
d["geocode"]

{'cc': 'BR',
 'center': {'lat': -22.31472, 'lng': -49.06056},
 'displayString': 'Bauru, SP, Brazil',
 'geometry': {'bounds': {'ne': {'lat': -22.140281999999996,
    'lng': -48.973036940248306},
   'sw': {'lat': -22.401756000000024, 'lng': -49.18743418680616}}},
 'longId': '72057594041398215',
 'slug': 'bauru-brazil',
 'what': '',
 'where': 'bauru'}

In [8]:
d["groups"][0].keys()


dict_keys(['type', 'name', 'items'])

In [9]:
d["groups"][0]["type"], d["groups"][0]["name"]

('Recommended Places', 'recommended')

In [10]:
items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]

number of items: 47


{'reasons': {'count': 0,
  'items': [{'reasonName': 'globalInteractionReason',
    'summary': 'This spot is popular',
    'type': 'general'}]},
 'referralId': 'e-4-4b9b9d3ef964a520b41336e3-0',
 'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/pub_',
     'suffix': '.png'},
    'id': '4bf58dd8d48988d116941735',
    'name': 'Bar',
    'pluralName': 'Bars',
    'primary': True,
    'shortName': 'Bar'}],
  'id': '4b9b9d3ef964a520b41336e3',
  'location': {'address': 'R. Benjamim Constant, 10-45',
   'cc': 'BR',
   'city': 'Bauru',
   'country': 'Brasil',
   'formattedAddress': ['R. Benjamim Constant, 10-45', 'Bauru, SP', 'Brasil'],
   'labeledLatLngs': [{'label': 'display',
     'lat': -22.32203285830864,
     'lng': -49.05903879065376}],
   'lat': -22.32203285830864,
   'lng': -49.05903879065376,
   'state': 'SP'},
  'name': 'Bar do Português',
  'photos': {'count': 0, 'groups': []}}}

In [15]:
items[1]

{'reasons': {'count': 0,
  'items': [{'reasonName': 'globalInteractionReason',
    'summary': 'This spot is popular',
    'type': 'general'}]},
 'referralId': 'e-4-4c40d6b1d691c9b615d58b0a-1',
 'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/pub_',
     'suffix': '.png'},
    'id': '4bf58dd8d48988d116941735',
    'name': 'Bar',
    'pluralName': 'Bars',
    'primary': True,
    'shortName': 'Bar'}],
  'id': '4c40d6b1d691c9b615d58b0a',
  'location': {'address': 'R. Antonio dos Reis',
   'cc': 'BR',
   'city': 'Bauru',
   'country': 'Brasil',
   'formattedAddress': ['R. Antonio dos Reis',
    'Bauru, SP',
    '17013-300',
    'Brasil'],
   'labeledLatLngs': [{'label': 'display',
     'lat': -22.32081804942013,
     'lng': -49.06109806793662}],
   'lat': -22.32081804942013,
   'lng': -49.06109806793662,
   'postalCode': '17013-300',
   'state': 'SP'},
  'name': 'Bar do Brecha',
  'photos': {'count': 0, 'groups': []}}}

In [11]:
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    print(location)
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    address =  ''
    if hasattr(location, 'address'):
      address = location['address']
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address", "postalcode", "lat", "lng"])
print("found %i bars" % len(df))
df.head()

{'address': 'R. Benjamim Constant, 10-45', 'lat': -22.32203285830864, 'lng': -49.05903879065376, 'labeledLatLngs': [{'label': 'display', 'lat': -22.32203285830864, 'lng': -49.05903879065376}], 'cc': 'BR', 'city': 'Bauru', 'state': 'SP', 'country': 'Brasil', 'formattedAddress': ['R. Benjamim Constant, 10-45', 'Bauru, SP', 'Brasil']}
{'address': 'R. Antonio dos Reis', 'lat': -22.32081804942013, 'lng': -49.06109806793662, 'labeledLatLngs': [{'label': 'display', 'lat': -22.32081804942013, 'lng': -49.06109806793662}], 'postalCode': '17013-300', 'cc': 'BR', 'city': 'Bauru', 'state': 'SP', 'country': 'Brasil', 'formattedAddress': ['R. Antonio dos Reis', 'Bauru, SP', '17013-300', 'Brasil']}
{'address': 'Rua Caetano Sampieri, 1-15', 'lat': -22.33762104221057, 'lng': -49.05886850314782, 'labeledLatLngs': [{'label': 'display', 'lat': -22.33762104221057, 'lng': -49.05886850314782}], 'cc': 'BR', 'city': 'Bauru', 'state': 'SP', 'country': 'Brasil', 'formattedAddress': ['Rua Caetano Sampieri, 1-15', 

,uid,name,shortname,address,postalcode,lat,lng
0,4c40d6b1d691c9b615d58b0a,Bar do Brecha,Bar,,17013-300,-22.320818,-49.061098
1,4ce095e6df986ea85d71e816,Armazén Bar,Rock Club,,17017-290,-22.327265,-49.080345
2,4d680dc68f70a09368731a35,Bar da Rosa,Bar,,17012-170,-22.328441,-49.063031
3,520050da498e7ac981f1f51f,Quiosque Chopp Brahma,Brewery,,17013-113,-22.316145,-49.066066
4,51548e26e4b044d18bda1f6f,SAMPA 27-28,Bar,,17012-060,-22.336280,-49.066263


In [12]:
bauru_center = d["geocode"]["center"]
bauru_center

{'lat': -22.31472, 'lng': -49.06056}

In [13]:
from folium import plugins


map_Bauru = folium.Map(location=[-22.31472, -49.06056], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='red',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_Bauru)

add_markers(df)
hm_data = df[["lat", "lng"]].to_numpy().tolist()
map_Bauru.add_child(plugins.HeatMap(hm_data))

map_Bauru

In [22]:
lat = -22.31472
lng = -49.06056
map_Bauru = folium.Map(location=[lat, lng], zoom_start=17)
add_markers(df)
folium.CircleMarker(
    [lat, lng],
    radius=15,
    popup="Best Point!",
    color='green',
    fill=True,
    fill_color='#0000FF',
    fill_opacity=0.7,
    parse_html=False).add_to(map_Bauru)
map_Bauru